In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import re
import numpy as np
import pandas as pd
import pickle
from mecab import MeCab

In [12]:
#Hyperparameter

hidden_size = 256
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2
UNK_TOKEN = 3
MAX_LENGTH = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
def clean_text(text):
    if pd.isna(text): # NaN 값 처리
        return ''
    text = text.lower()
    text = re.sub(r'\d+', ' ', text) # 숫자는 공백으로 처리
    text = re.sub(r'([^\w\s])', r' \1 ', text) # 마침표 앞 뒤로 공백 추가
    text = re.sub(r'\s+', ' ', text) # 두 개 이상의 공백을 하나로 처리
    text = text.strip() # 텍스트 양 옆의 공백 제거
    
    return text

In [14]:
def indiceFromSentence(vocab, sentence):
    return [vocab.get(word, vocab['<UNK>']) for word in sentence.split(' ')]

In [16]:
def tensorFromSentence(vocab, sentence):
    indice = indiceFromSentence(vocab, sentence)
    indice.append(EOS_TOKEN)
    return torch.tensor(indice, dtype=torch.long, device=device).view(-1, 1)

In [17]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2)
        
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden
    
    def initHidden(self):
        return(torch.zeros(2, 1, self.hidden_size, device=device), torch.zeros(2, 1, self.hidden_size, device=device))

In [ ]:
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.out(output[0])
        return output, hidden
    
    def initHidden(self):
        return(torch.zeros(2, 1, self.hidden_size, device=device), torch.zeros(2, 1, self.hidden_size, device=device))

In [ ]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    loss = 0
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        
    decoder_input = torch.tensor([[SOS_TOKEN]], device=device)
    decoder_hidden = encoder_hidden
    
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()
        loss += criterion(decoder_output, target_tensor[di])
        
        if decoder_input.item() == EOS_TOKEN:
            break
        
    loss.backward() # 역전파 
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length
    
    

In [20]:
def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01):
    print_loss_total = 0
    
    for iter in range(1, n_iters+1):
        training_pair = random.choice(pairs) # input - target pair
        input_tensor = tensorFromSentence(word_to_idx, training_pair[0]).to(device)
        target_tensor = tensorFromSentence(word_to_idx, training_pair[1]).to(device)
        
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        
        if iter % print_every == 0:
            print_lost_avg = print_loss_total / print_every
            print(f'Iteration : {iter}, Loss : {print_lost_avg: .4f}')
            print_loss_total = 0

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(word_to_idx, sentence).to(device)
        input_length = input_tensor.size(0)
        encoder_hidden = encoder.initHidden()
        encoder_hidden = tuple([e.to(device) for e in encoder_hidden])
        
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder_hidden(input_tensor[ei], encoder_hidden)
            
        decoder_input = torch.tensor([[SOS_TOKEN]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = [] # output sentence
        
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_TOKEN:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(idx_to_word[topi.item()]) # 최종 아웃풋의 index
            
            decoder_input = topi.squeeze().detach()
        
        return ' '.join(decoded_words)

In [22]:
def chat(encoder, decoder, max_length=MAX_LENGTH):
    print("Let's chat (type 'bye' to exit)")
    while True:
        input_sentence = input(">>")
        if input_sentence == 'bye':
            break
        output_sentence = evaluate(encoder, decoder, input_sentence)
        print('<', output_sentence)

In [23]:
# load data and preprocessing
df = pd.read_csv('./dementia_fix.csv', sep=',', names=['Question', 'Intention', 'Answer'], skiprows=1)
df['Encoder Inputs'] = df['Question'].apply(clean_text)
df['Decoder Inputs'] = df['Answer'].apply(clean_text)


In [24]:
df['Encoder Inputs']

0       알츠하이머병의 원인으로 매일 소주를 섭취하는 것이 언급되고 있는데 , 이에 대한 근...
1                         알츠하이머병이라는 질병은 유전적 영향을 받는 것인가요 ?
2                     알츠하이머병의 발생 원인에 대한 연구나 발견이 진행 중인가요 ?
3              알츠하이머병의 발병과 관련하여 뇌의 노화로 인한 증상과 원인을 알려주세요 .
4                   알츠하이머병의 원인과 관련된 연구 결과가 있을까요 ? 알려주세요 .
                              ...                        
6618                         치매 치료에는 어떤 운동이나 작업이 효과적일까요 ?
6619    치매 치료의 결과와 과정을 상세히 설명해주세요 . 치매 치료의 효과는 어떻게 나타날...
6620                      치매를 치료하기 위해 어떤 치료 방법들이 효과적일까요 ?
6621                        치매 치료를 위해 어떤 약물이 사용될 수 있을까요 ?
6622                         치매 치료를 위해 어떤 전문가와 협력해야 할까요 ?
Name: Encoder Inputs, Length: 6623, dtype: object

In [25]:
df['Decoder Inputs']

0       알츠하이머병의 정확한 원인은 아직 밝혀지지 않았지만 , 연구들이 알츠하이머병의 발병...
1       알츠하이머병은 현재까지 완전한 원인이 밝혀지지 않았습니다 . 알츠하이머병은 아직 완...
2       알츠하이머병은 치매를 일으키는 가장 흔한 퇴행성 뇌질환으로 , 년 독일 의사 알로이...
3       알츠하이머병은 현재까지 그 발병 원인에 대한 완벽한 해명은 아직 이루어지지 않았습니...
4       알츠하이머병은 복잡한 질환으로 , 아직도 원인이 완전히 밝혀진 것은 아닙니다 . 그...
                              ...                        
6618    치매는 노인들에게 주로 발생하는 뇌질환으로 , 원인과 치료 방법은 아직 완전히 밝혀...
6619    치매는 일상 생활을 수행하는 능력을 심각하게 손상시키는 질환으로 , 후천성 치매와 ...
6620    치매는 노화로 인해 기억력과 지능을 점차적으로 잃는 질병으로 , 알츠하이머병이 주요...
6621    알츠하이머병은 뇌에 변화가 생겨서 인지 기능에 장애가 생기는 신경퇴행성 질환입니다 ...
6622    치매는 현재까지 완전한 치료가 불가능한 치매입니다 . 치매는 다양한 원인에 의해 발...
Name: Decoder Inputs, Length: 6623, dtype: object

In [26]:
input_sentence = [sentence for sentence in df['Encoder Inputs']]
output_sentence = [sentence + "<EOS>" for sentence in df['Decoder Inputs']]

In [27]:
input_sentence[0:5]

['알츠하이머병의 원인으로 매일 소주를 섭취하는 것이 언급되고 있는데 , 이에 대한 근거가 있는지 알려주세요 .',
 '알츠하이머병이라는 질병은 유전적 영향을 받는 것인가요 ?',
 '알츠하이머병의 발생 원인에 대한 연구나 발견이 진행 중인가요 ?',
 '알츠하이머병의 발병과 관련하여 뇌의 노화로 인한 증상과 원인을 알려주세요 .',
 '알츠하이머병의 원인과 관련된 연구 결과가 있을까요 ? 알려주세요 .']

In [28]:
output_sentence[0:5]

['알츠하이머병의 정확한 원인은 아직 밝혀지지 않았지만 , 연구들이 알츠하이머병의 발병 기전에 대해 논의하고 있습니다 . 일부 연구에 따르면 , 유전적인 요소와 뇌의 기능 손상이 관련되어 있다고 알려져 있습니다 . 알츠하이머병은 아밀로이드 베타 단백질과 타우 단백질의 과도한 생성 , 뇌 세포의 비정상적인 활동 , 뇌 조직의 변화로 인해 발생하는 것으로 생각되고 있습니다 . 이러한 변화가 알츠하이머병의 발병 위험을 증가시키고 , 병의 진행을 가속화시킨다는 것입니다 . 알츠하이머병의 발병과 관련된 위험 요소에 대해서는 더 많은 연구와 조사가 필요합니다 . 더 많은 연구와 자료 수집을 통해 알츠하이머병에 대한 더 많은 이해와 예방 방법이 개발될 것으로 기대됩니다 .<EOS>',
 '알츠하이머병은 현재까지 완전한 원인이 밝혀지지 않았습니다 . 알츠하이머병은 아직 완전히 이해되지 않았지만 , 연구 결과에 따르면 유전적인 요소와 다양한 환경적인 요인이 이 질환을 일으키는 역할을 한다고 알려져 있습니다 . 특히 , 아밀로이드 베타 단백질의 비정상적인 축적이 알츠하이머병과 관련이 있는 것으로 알려져 있습니다 . 이 외에도 나이 , 노화 , 고혈압 , 당뇨병 , 그리고 흡연 등과 같은 다른 요인들도 알츠하이머병 발병과 연관성이 있을 수 있습니다 . 더 많은 연구와 조사를 통해 알츠하이머병의 원인을 파악하고 예방 방법을 개발할 필요가 있습니다 .<EOS>',
 '알츠하이머병은 치매를 일으키는 가장 흔한 퇴행성 뇌질환으로 , 년 독일 의사 알로이스 알츠하이머에 의해 처음으로 보고되었습니다 . 이 질환의 원인에 대해서는 현재까지 명확한 답은 없으나 , 치매 발생의 위험 요소와 관련하여 몇 가지 위험 요인이 알려져 있습니다 . 일반적으로 , 가장 잘 알려진 요인 중 하나는 고령입니다 . 고령은 치매의 발병 위험을 증가시키는 가장 큰 위험 요소로 알려져 있습니다 . 또한 , 가족력이 있는 경우 알츠하이머병 발생 위험이 높아집니다 . 연구에 따르면 , 조발성 가족성 알츠하이머병은 주로 

In [65]:
# 단어 사전 생성
all_word = set(' '.join(df['Encoder Inputs'].tolist() + df['Decoder Inputs'].tolist()).split())
vocab = {'<PAD>': PAD_TOKEN, '<SOS>': SOS_TOKEN, '<EOS>': EOS_TOKEN, '<UNK>': UNK_TOKEN}
vocab.update({word: i+4 for i, word in enumerate(all_word)})
vocab_size = len(vocab)

with open('vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

In [66]:
all_word

{'산책하기',
 '직업',
 'mr은',
 '이완에',
 '특정',
 '정확히',
 '이행될',
 '혼동할',
 '시도하더라도',
 '건강에',
 '보여주며',
 '불쾌감과',
 '지구력을',
 '기력이',
 '개별화되어야',
 '그렇다면',
 '이루고',
 '우리는',
 '진료과에서도',
 '일상생활이',
 '있어야만',
 '수면의',
 '결과에서는',
 '일자리의',
 '쓰기에도',
 '사용량',
 '군단요법과',
 '풀어',
 '시험은',
 '기대',
 '백해무익으로',
 '스트레스도',
 '우리',
 '기저신경핵이',
 '효과와의',
 '기분과는',
 '신체의',
 '연구결과와',
 '되기도',
 '운동이지만',
 '손동작',
 '추론을',
 '헌팅톤병',
 '갑작스럽게',
 '이상입니다',
 '예측하여',
 '준수하며',
 '것인가요',
 '복잡합니다',
 '구하여',
 '않아서',
 '우울장애를',
 '장애를',
 '공유하고',
 '증가하는데',
 'ssri로',
 '최소한의',
 '매뉴얼',
 '적적',
 '나타나면',
 '피하는',
 '기술을',
 '제대로',
 '시행해도',
 '임상에서',
 '식품은',
 '수행능력',
 '의존증과',
 '어긋나',
 '연구들에',
 '상실시킵니다',
 '조사도',
 '일반인들도',
 '보여주어',
 '뇌신경계와의',
 '에는',
 '사람들에서',
 '조정하는',
 '독립성을',
 '활동만으로는',
 '이끌어내기도',
 '들어가고',
 '간주되며',
 '인격장애는',
 '합병증도',
 '네트워크의',
 '심각해진다면',
 '연구에는',
 '소화될',
 '강화하며',
 '과업을',
 '뇌혈관질환으로',
 '확장되고',
 '지원에',
 '기르며',
 '간주될',
 '뇌활성제',
 '인격',
 '목적은',
 '양뿐만',
 '수축을',
 '부족해지는',
 '공감하는',
 '교류에도',
 '원인이므로',
 '경향도',
 '만성화되어',
 '이르고',
 '상관',
 '우울증에',
 '먹어도',
 '언어장애가',
 '신질

In [67]:
vocab

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<UNK>': 3,
 '산책하기': 4,
 '직업': 5,
 'mr은': 6,
 '이완에': 7,
 '특정': 8,
 '정확히': 9,
 '이행될': 10,
 '혼동할': 11,
 '시도하더라도': 12,
 '건강에': 13,
 '보여주며': 14,
 '불쾌감과': 15,
 '지구력을': 16,
 '기력이': 17,
 '개별화되어야': 18,
 '그렇다면': 19,
 '이루고': 20,
 '우리는': 21,
 '진료과에서도': 22,
 '일상생활이': 23,
 '있어야만': 24,
 '수면의': 25,
 '결과에서는': 26,
 '일자리의': 27,
 '쓰기에도': 28,
 '사용량': 29,
 '군단요법과': 30,
 '풀어': 31,
 '시험은': 32,
 '기대': 33,
 '백해무익으로': 34,
 '스트레스도': 35,
 '우리': 36,
 '기저신경핵이': 37,
 '효과와의': 38,
 '기분과는': 39,
 '신체의': 40,
 '연구결과와': 41,
 '되기도': 42,
 '운동이지만': 43,
 '손동작': 44,
 '추론을': 45,
 '헌팅톤병': 46,
 '갑작스럽게': 47,
 '이상입니다': 48,
 '예측하여': 49,
 '준수하며': 50,
 '것인가요': 51,
 '복잡합니다': 52,
 '구하여': 53,
 '않아서': 54,
 '우울장애를': 55,
 '장애를': 56,
 '공유하고': 57,
 '증가하는데': 58,
 'ssri로': 59,
 '최소한의': 60,
 '매뉴얼': 61,
 '적적': 62,
 '나타나면': 63,
 '피하는': 64,
 '기술을': 65,
 '제대로': 66,
 '시행해도': 67,
 '임상에서': 68,
 '식품은': 69,
 '수행능력': 70,
 '의존증과': 71,
 '어긋나': 72,
 '연구들에': 73,
 '상실시킵니다': 74,
 '조사도': 75,
 '일반인들도': 76,
 '보여주어': 77,
 '뇌신경

In [68]:
word_to_idx = vocab
idx_to_word = {i: word for word, i in word_to_idx.items()}

In [69]:
word_to_idx

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<UNK>': 3,
 '산책하기': 4,
 '직업': 5,
 'mr은': 6,
 '이완에': 7,
 '특정': 8,
 '정확히': 9,
 '이행될': 10,
 '혼동할': 11,
 '시도하더라도': 12,
 '건강에': 13,
 '보여주며': 14,
 '불쾌감과': 15,
 '지구력을': 16,
 '기력이': 17,
 '개별화되어야': 18,
 '그렇다면': 19,
 '이루고': 20,
 '우리는': 21,
 '진료과에서도': 22,
 '일상생활이': 23,
 '있어야만': 24,
 '수면의': 25,
 '결과에서는': 26,
 '일자리의': 27,
 '쓰기에도': 28,
 '사용량': 29,
 '군단요법과': 30,
 '풀어': 31,
 '시험은': 32,
 '기대': 33,
 '백해무익으로': 34,
 '스트레스도': 35,
 '우리': 36,
 '기저신경핵이': 37,
 '효과와의': 38,
 '기분과는': 39,
 '신체의': 40,
 '연구결과와': 41,
 '되기도': 42,
 '운동이지만': 43,
 '손동작': 44,
 '추론을': 45,
 '헌팅톤병': 46,
 '갑작스럽게': 47,
 '이상입니다': 48,
 '예측하여': 49,
 '준수하며': 50,
 '것인가요': 51,
 '복잡합니다': 52,
 '구하여': 53,
 '않아서': 54,
 '우울장애를': 55,
 '장애를': 56,
 '공유하고': 57,
 '증가하는데': 58,
 'ssri로': 59,
 '최소한의': 60,
 '매뉴얼': 61,
 '적적': 62,
 '나타나면': 63,
 '피하는': 64,
 '기술을': 65,
 '제대로': 66,
 '시행해도': 67,
 '임상에서': 68,
 '식품은': 69,
 '수행능력': 70,
 '의존증과': 71,
 '어긋나': 72,
 '연구들에': 73,
 '상실시킵니다': 74,
 '조사도': 75,
 '일반인들도': 76,
 '보여주어': 77,
 '뇌신경

In [70]:
idx_to_word

{0: '<PAD>',
 1: '<SOS>',
 2: '<EOS>',
 3: '<UNK>',
 4: '산책하기',
 5: '직업',
 6: 'mr은',
 7: '이완에',
 8: '특정',
 9: '정확히',
 10: '이행될',
 11: '혼동할',
 12: '시도하더라도',
 13: '건강에',
 14: '보여주며',
 15: '불쾌감과',
 16: '지구력을',
 17: '기력이',
 18: '개별화되어야',
 19: '그렇다면',
 20: '이루고',
 21: '우리는',
 22: '진료과에서도',
 23: '일상생활이',
 24: '있어야만',
 25: '수면의',
 26: '결과에서는',
 27: '일자리의',
 28: '쓰기에도',
 29: '사용량',
 30: '군단요법과',
 31: '풀어',
 32: '시험은',
 33: '기대',
 34: '백해무익으로',
 35: '스트레스도',
 36: '우리',
 37: '기저신경핵이',
 38: '효과와의',
 39: '기분과는',
 40: '신체의',
 41: '연구결과와',
 42: '되기도',
 43: '운동이지만',
 44: '손동작',
 45: '추론을',
 46: '헌팅톤병',
 47: '갑작스럽게',
 48: '이상입니다',
 49: '예측하여',
 50: '준수하며',
 51: '것인가요',
 52: '복잡합니다',
 53: '구하여',
 54: '않아서',
 55: '우울장애를',
 56: '장애를',
 57: '공유하고',
 58: '증가하는데',
 59: 'ssri로',
 60: '최소한의',
 61: '매뉴얼',
 62: '적적',
 63: '나타나면',
 64: '피하는',
 65: '기술을',
 66: '제대로',
 67: '시행해도',
 68: '임상에서',
 69: '식품은',
 70: '수행능력',
 71: '의존증과',
 72: '어긋나',
 73: '연구들에',
 74: '상실시킵니다',
 75: '조사도',
 76: '일반인들도',
 77: '보여주어',
 78: 

In [71]:
word_to_idx['이루고']

20

In [72]:
idx_to_word[20]

'이루고'

In [73]:
encoder = EncoderLSTM(vocab_size, hidden_size).to(device)
decoder = DecoderLSTM(hidden_size, vocab_size).to(device)

In [74]:
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.005)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

In [75]:
pairs = [list(x) for x in zip(df['Encoder Inputs'], df['Decoder Inputs'])]

In [76]:
pairs[1]

['알츠하이머병이라는 질병은 유전적 영향을 받는 것인가요 ?',
 '알츠하이머병은 현재까지 완전한 원인이 밝혀지지 않았습니다 . 알츠하이머병은 아직 완전히 이해되지 않았지만 , 연구 결과에 따르면 유전적인 요소와 다양한 환경적인 요인이 이 질환을 일으키는 역할을 한다고 알려져 있습니다 . 특히 , 아밀로이드 베타 단백질의 비정상적인 축적이 알츠하이머병과 관련이 있는 것으로 알려져 있습니다 . 이 외에도 나이 , 노화 , 고혈압 , 당뇨병 , 그리고 흡연 등과 같은 다른 요인들도 알츠하이머병 발병과 연관성이 있을 수 있습니다 . 더 많은 연구와 조사를 통해 알츠하이머병의 원인을 파악하고 예방 방법을 개발할 필요가 있습니다 .']

In [77]:
trainIters(encoder, decoder, 10000, 100)

Iteration : 100, Loss :  7.7037
Iteration : 200, Loss :  7.1107
Iteration : 300, Loss :  6.9821


KeyboardInterrupt: 

In [ ]:
encoder.eval()
decoder.eval()